In [1]:
import feedparser
import csv
import pandas as pd
import re

##### Indicate dictionary with RSS-feeds, be care, not all of  RSS have good format and could be parsed, check them by one better
##### Indicate two vectors to search with certain words, symbol '|' means "OR", if you need only one vector, in end of second one put '|' - it is mean, that vector will search all words. Vectors not sensability to register
##### Indicate two path to files: file with all news, file with news by certain words

In [2]:
newsurls = {'NYTimes': 'https://rss.nytimes.com/services/xml/rss/nyt/World.xml',
            'ChicagoTribune' : 'https://www.chicagotribune.com/arcio/rss/category/news/breaking/?query=display_date:%5Bnow-2d+TO+now%5D+AND+revision.published:true&sort=display_date:desc#nt=instory-link'
} #example of RSS feeds
                                           
f_all_news = 'allnews23march20.csv'
f_certain_news = 'certainnews23march20.csv'

vector1 = 'pubs|restaraunts' #example of targets (vectors)
vector2 = 'close|open'

In [3]:
def parseRSS( rss_url ): #function gets link of RSS, return parsed feed with feedpaeser library
    return feedparser.parse( rss_url )  
    
def getHeadlines( rss_url ): #function to get headlines of news
    headlines = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        headlines.append(newsitem['title'])
    return headlines

def getDescriptions( rss_url ): #function to get description of news
    descriptions = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        descriptions.append(newsitem['description'])
    return descriptions

def getLinks( rss_url ): #function to get links of news
    links = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        links.append(newsitem['link'])
    return links

def getDates( rss_url ): #function to get dates of news
    dates = []
    feed = parseRSS( rss_url )
    for newsitem in feed['items']:
        dates.append(newsitem['published'])
    return dates

In [4]:
allheadlines = []
alldescriptions = []
alllinks = []
alldates = []
# iterate our URLs and adding results to empty lists
for key,url in newsurls.items():
    allheadlines.extend( getHeadlines( url ) )
    
for key,url in newsurls.items():
    alldescriptions.extend( getDescriptions( url ) )
    
for key,url in newsurls.items():
    alllinks.extend( getLinks( url ) )
    
for key,url in newsurls.items():
    alldates.extend( getDates( url ) )

In [5]:
def write_all_news(all_news_filepath): #function to writing all of news to .csv, return this dataset
    header = ['Title','Description','Links','Publication Date'] 

    with open(all_news_filepath, 'w', encoding='utf-8-sig') as csvfile:
        writer = csv.writer(csvfile, delimiter=',')

        writer.writerow(i for i in header) 

        for a,b,c,d  in zip(allheadlines,alldescriptions,
                            alllinks, alldates):
            writer.writerow((a,b,c,d))


        df = pd.read_csv(all_news_filepath)
            
    return df

def looking_for_certain_news(all_news_filepath, certain_news_filepath, target1, target2): #function for search, then writing
                                                                                         #certain news by targeting vectors,
                                                                                        #then return dataset with certain news
    df = pd.read_csv(all_news_filepath)
    
    result = df.apply(lambda x: x.str.contains(target1, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    result2 = df.apply(lambda x: x.str.contains(target2, na=False,
                                    flags = re.IGNORECASE, regex=True)).any(axis=1)
    new_df = df[result&result2]
        
    new_df.to_csv(certain_news_filepath
                     ,sep = '\t', encoding='utf-8-sig')
        
    return new_df

In [6]:
write_all_news(f_all_news) #all news

,Title,Description,Links,Publication Date
0,Coronavirus Live Updates: Global Leaders Beg P...,Spain’s health system is under strain as medic...,https://www.nytimes.com/2020/03/23/world/coron...,"Mon, 23 Mar 2020 16:53:31 +0000"
1,Germany Bans Groups of More Than 2 to Stop Cor...,Chancellor Angela Merkel went into isolation a...,https://www.nytimes.com/2020/03/22/world/europ...,"Sun, 22 Mar 2020 20:35:38 +0000"
2,"As Coronavirus Surveillance Escalates, Persona...",Tracking entire populations to combat the pand...,https://www.nytimes.com/2020/03/23/technology/...,"Mon, 23 Mar 2020 13:04:38 +0000"
3,Video Chat With Taliban Keeps Afghan Peace Dea...,Trying to iron out a disagreement over a priso...,https://www.nytimes.com/2020/03/23/world/asia/...,"Mon, 23 Mar 2020 16:10:25 +0000"
4,"Italy, Pandemic’s New Epicenter, Has Lessons f...",The country’s experience shows that steps to i...,https://www.nytimes.com/2020/03/21/world/europ...,"Sun, 22 Mar 2020 08:05:52 +0000"
...,...,...,...,...
83,Curbside critic: The Foxtail carryout menu off...,Curbside critic: Phil Vettel reviews the carry...,https://www.chicagotribune.com/coronavirus/ct-...,"Sun, 22 Mar 2020 14:00:00 +0000"
84,"2 shot, 1 fatally, overnight throughout Chicag...",A man was killed in the city’s first shooting ...,https://www.chicagotribune.com/news/breaking/c...,"Sun, 22 Mar 2020 12:00:00 +0000"
85,Burglars smash through glass door of Lincoln P...,A group of people in dark clothing broke into ...,https://www.chicagotribune.com/news/breaking/c...,"Sun, 22 Mar 2020 10:10:29 +0000"
86,"For the first time, all Peace Corps volunteers...","More than 7,000 Peace Corps volunteers in 60 c...",https://www.chicagotribune.com/coronavirus/ct-...,"Sun, 22 Mar 2020 10:00:00 +0000"


In [7]:
looking_for_certain_news(f_all_news, f_certain_news, vector1, vector2) #certain news by vectors

,Title,Description,Links,Publication Date
48,"Johnson Orders British Pubs to Close, Ending a...",The prime minister’s reluctant action effectiv...,https://www.nytimes.com/2020/03/20/world/europ...,"Fri, 20 Mar 2020 23:49:35 +0000"
